In [ ]:
import langchain


In [1]:
from langchain.agents import initialize_agent, Tool

from dotenv import load_dotenv
import os


load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
# !pip install -U duckduckgo-search
from langchain_google_genai import GoogleGenerativeAI

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

# Web Search Tool using DuckDuckGo
search_tool = Tool(
    name="DuckDuckGo Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches using DuckDuckGo."
)



from langchain_groq import ChatGroq

llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=groq_api_key) 
tools = [search_tool]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("What are the top trending movies this week in india ?")

print(response)

/var/folders/fd/dk1cn0wn0j3b7_0hf6vgbgp40000gn/T/ipykernel_57427/853456106.py:20: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
/var/folders/fd/dk1cn0wn0j3b7_0hf6vgbgp40000gn/T/ipykernel_57427/853456106.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed 



> Entering new AgentExecutor chain...
To find the top trending movies this week in India, I need to search for the latest information on the internet. The best way to do this would be to use a search engine that can provide me with the most recent and relevant results.

Action: DuckDuckGo Search
Action Input: "top trending movies this week in India"

(let's see what the search results say)
Observation: 10 Yesterday 9 Feb 9 07 This week 06 Last week 2 This month 1 Last month 25 This year 24 Last year 23 2023 22 2022 ... Top Movies and TV Shows on Netflix in India on February 10, 2025 by day. Top ranked movies. week Mon Tue Wed Thu Fri Sat Sun; 07: ... These movie charts are based on the official trending or the most popular movies on VOD. In Vicky Vidya Ka Woh Wala Video, newlyweds Vicky (Rajkummar Rao) and Vidya (Tripti Dimri) set out on a frantic search after they misplace a video of their first night together.Set in 1997, this comedy-drama is a fun ride that is now available to str

In [3]:
# The WikipediaAPIWrapper - allows you to query Wikipedia for articles and summaries.

#!pip install wikipedia

from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

# Step 1: Initialize the Wikipedia Tool
wikipedia_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search Wikipedia for information on any topic."
)

# llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.5)

llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=groq_api_key) 

tools = [wikipedia_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

query = "Who is Narendra Modhi?"
response = agent.run(query)
print("Response from Agent:")
print(response)



> Entering new AgentExecutor chain...
To find information on Narendra Modhi, I should first search for his name on Wikipedia to get an overview of who he is and what he is known for.

Action: Wikipedia Search
Action Input: Narendra Modhi
Observation: No good Wikipedia Search Result was found
Thought:Since the search for "Narendra Modhi" didn't yield any results, I should try searching with a slightly different spelling, as it's possible that the name is spelled differently or there's a minor typo.

Action: Wikipedia Search
Action Input: Narendra Modi
Observation: Page: Narendra Modi
Summary: Narendra Damodardas Modi (born 17 September 1950) is an Indian politician who has served as the prime minister of India since 2014. Modi was the chief minister of Gujarat from 2001 to 2014 and is the member of parliament (MP) for Varanasi. He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindu nationalist paramilitary volunteer organisa

In [4]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")


Database created and populated with sample data.


In [5]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

In [6]:
# Step 1: Connect to the Database
db = SQLDatabase.from_uri("sqlite:///example.db")

# Step 3: Initialize the LLM
# llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.5)
llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=groq_api_key) 


# Step 2: Create the SQL Database Toolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("Response:", response1)



> Entering new AgentExecutor chain...
To find out how many units of Product A are in stock, I need to know the structure of the tables in the database, specifically the ones related to products and inventory. 

Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought:Since I have the list of tables, which includes "inventory", my next step is to understand the schema of the "inventory" table to see what fields are available.

Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Product A	150
2	Product B	200
3	Product C	300
*/
Thought:With the schema of the "inventory" table, I can see that it has fields for "id", "product_name", and "stock_quantity". To find out how many units of Product A are in stock, I need to query the "inventory" table for the row where "product_name" i

In [7]:
# ArxivQueryRun
from langchain.agents import initialize_agent, Tool

from dotenv import load_dotenv
import os

load_dotenv()
# google_api_key = os.getenv("GOOGLE_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

from langchain_community.tools.arxiv.tool import ArxivQueryRun

arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)

# Step 2: Initialize the LLM
# llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.5)
llm = ChatGroq(model_name="llama-3.3-70b-versatile", groq_api_key=groq_api_key) 

# Step 3: Combine the tool into a list
tools = [arxiv_tool]

# Step 4: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Step 5: Query the Agent
query = "Search for research papers on Diffusion Models in 2024."
response = agent.run(query)
print("Response from Agent:")
print(response)




> Entering new AgentExecutor chain...
To find research papers on Diffusion Models in 2024, I should use the Arxiv Search tool with relevant keywords.

Action: Arxiv Search
Action Input: {'query': 'Diffusion Models', 'date': '2024'}

(I'll wait for the result...)
Observation: No good Arxiv Result was found
Thought:Since the initial search didn't yield any good results, I should try to refine my search query to get more relevant results.

Action: Arxiv Search
Action Input: {'query': 'Diffusion Models 2024', 'date': '2024', 'tags': ['machine learning', 'deep learning']}

(I'll wait for the result...)
Observation: No good Arxiv Result was found
Thought:Since the previous searches didn't yield any good results, I should try to use more specific keywords related to Diffusion Models and also use authors who are known to work in this area.

Action: Arxiv Search
Action Input: {'query': 'Diffusion Models for Image Generation 2024', 'date': '2024', 'tags': ['machine learning', 'deep learning', 

In [15]:
## With Groq APi KEy

In [8]:
# Import necessary libraries
from langchain.agents import initialize_agent, Tool, AgentType
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")  # Corrected variable name

# Import Arxiv Search Tool
from langchain_community.tools.arxiv.tool import ArxivQueryRun

arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)

# Step 2: Initialize the LLM
from langchain_groq import ChatGroq

llm = ChatGroq(model_name="llama3-8b-8192", groq_api_key=groq_api_key)  # Corrected API key usage

# Step 3: Combine the tool into a list
tools = [arxiv_tool]

# Step 4: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Corrected agent type
    verbose=False
)

# Step 5: Query the Agent
query = "Search for research papers on Diffusion Models in 2024."
response = agent.run(query)

# Print the response
print("Response from Agent:")
print(response)


Response from Agent:
There are no research papers specifically on Diffusion Models in 2024.


In [9]:
from langchain_community.tools.file_management import ReadFileTool
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

from dotenv import load_dotenv
import os

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
# Step 1: Initialize the ReadFileTool
read_file_tool = Tool(
    name="Read File",
    func=ReadFileTool().run,
    description="Use this tool to read the content of a text file in the current working directory. Provide the file name as input."
)

# Step 2: Initialize the LLM
llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.5)

# Step 3: Combine the tool into a list
tools = [read_file_tool]

# Step 4: Initialize the Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# Step 5: Query the Agent with a file name
file_name = "example.txt"

response = agent.run(f"Read the content of the file named {file_name}.")
print("Response from Agent:")
print(response)




> Entering new AgentExecutor chain...
Action: Read File
Action Input: example.txt
Observation: Hy Welcome to my Txt File
Thought:

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Final Answer: Hy Welcome to my Txt File

> Finished chain.
Response from Agent:
Hy Welcome to my Txt File
